# Data handling for movies 

In cognitive neuroscience, what was considered impossible a few decades ago is now doable: Today, we can infer certain aspects of a person's cognitive processes from measurements of brain activity. This progress has come about due to a confluence of improvements in three different areas: computing speeds, brain imaging methods, and efficient machine learning algorithms. To use all three of these aspects for our data analysis, in an optimal manner, involves learning a complex sequence of steps. Our overarching goal for the problem sets in this course is to provide a step-by-step walk-through on how to execute these analysis steps and infer cognitive states from brain activity.

This process begins with running experiments and collecting fMRI data. Then, collected data undergo preprocessing, which involves motion correction, cropping, transformations, and alignments. Only after these steps have been completed are the data ready for analysis by machine learning algorithms. Preprocessed fMRI data are commonly stored in the NIfTI format, the starting point for our analyses.

In this notebook, our aim is to cover the very first step in the analysis after preprocessing: the extraction and normalization, and masking of pre-processed fMRI data. We will also do a basic analysis which will show you the potential of movie based analyses

We will use our *first* movie dataset, called **Sherlock** from Chen et al. (2017). You can see more info about it in the [section](#movie) below.

### Goal of this notebook
This notebook will teach you how to visualize and normalize your data. Specifically, you will learn how to do the following:
 1. Load fMRI data into Python.
 2. Visualize fMRI data
 3. Plot the timeseries for a voxel
 4. Learn how masking volumes works
 5. Assess the homotopic activity during movie watching

## Table of Contents

[1. Import necessary packages](#import)  

[2. Introduction to the dataset](#movie)

[3. BIDS formatted data](#bids)

[4. Load fMRI](#load_fmri)  

[5. Masking data](#masking)

[6. Homotopic analyses](#homotopy)


### Exercises
>[Exercise 1](#ex1)   [2](#ex2)  [3](#ex3)  [4](#ex4)  [5](#ex5)  [6](#ex6)  [7](#ex7)  [8](#ex8)  [9](#ex9)  
>[Novel contribution](#novel) 


## 1. Import necessary packages <a id="import"></a> 

Like we covered last week, we need to import the functions we will use. In the script today, we are importing from a script called `utils.py` (stored in your git repo) instead of spelling out the packages we will use. The utils file contains 3 types of information:
1. The packages we will import. The main ones are listed below  
> numpy: Perform numerical operations in Python.  
> nibabel: Read  fMRI data into Python arrays.   
> scipy: Scientific computing methods.  
> nilearn: Used to extract masked fMRI data from a NIfTI file.  
> sklearn: Machine learning methods.  
2. Variables that are used often, like the paths to the data directories  
3. Functions that are used often  

In [ ]:
# We need to add the utils to the path
import sys
sys.path.insert(0, '..')

# We want to import the contents of utils so that all of the variables are available (rather than accessible by utils.X).
# "import *" does just that, it imports everything into the base
from utils import * 

# Specify some jupyter magics. Specifcally, make plots non-interactive and save regularly
%matplotlib inline 
%autosave 5

**Self-study:** You can navigate to the `utils` file in your jupyter notebook and read it in your browser. You should do that to get a feel for what it contains!

<div class="alert alert-block alert-info">
Sometimes we will have <strong>self-study</strong> sections that we encourage you to pursue ideas or read documents. These are intended to flesh out your understanding of the material and are encouraged.
</div>

We can look at the data directory for Sherlock, where we will pull data from

In [ ]:
print('Data dir = %s' % (sherlock_dir))

**Exercise 1:**<a id="ex1"></a> Describe the difference in functionality between `import numpy` and `from numpy import zeros`:

**A:**


## 2. Introduction to Sherlock <a id="movie"></a> 

When you collect your own data, you will likely know it intimately; however, when you're analyzing data that someone else has collected, it can be easy to lose the details. The Sherlock dataset is one we will come back to multiple times throughout this course, so it is important to know it well. 

This data was collected for the experiment reported in [Chen et al. (2017)](https://doi.org/10.1038/nn.4450). Subjects were watching the first hour of [A Study in Pink](https://en.wikipedia.org/wiki/A_Study_in_Pink), henceforth called "Sherlock". Below is the abstract: 

> Our lives revolve around sharing experiences and memories with others. When different people recount the same events, how similar are their underlying neural representations? Participants viewed a 50-min movie, then verbally described the events during functional MRI, producing unguided detailed descriptions lasting up to 40 min. As each person spoke, event-specific spatial patterns were reinstated in default-network, medial-temporal, and high-level visual areas. Individual event patterns were both highly discriminable from one another and similar among people, suggesting consistent spatial organization. In many high-order areas, patterns were more similar between people recalling the same event than between recall and perception, indicating systematic reshaping of percept into memory. These results reveal the existence of a common spatial organization for memories in high-level cortical areas, where encoded information is largely abstracted beyond sensory constraints, and that neural patterns during perception are altered systematically across people into shared memory representations for real-life events.

See a clip from the episode below. **Warning**: it contains gruesome content at 55s in. Watching it is entirely optional.

In [ ]:
# Typically we import all our functions at the top, but this one is pretty unique, so we are just doing it here.
from IPython.display import YouTubeVideo

YouTubeVideo('VaT7IYQgyqo')

The Sherlock data is stored in the `/farmshare/home/classes/psych/236/data/Sherlock/` folder.  This data was taken from [OpenNeuro](https://openneuro.org/datasets/ds001132/versions/00003) and so is stored in the [BIDS](https://bids.neuroimaging.io/), the current standard for MRI data storage (see below for more). In today's class, we are going to mainly focus on the `derivatives` folder, which contains data that has been preprocessed in useful ways. 

You have permission to view and read (AKA load) the files but you don't have permission to edit them. Hence, don't worry about altering the data through your analyses or exploration. 

The data has been preprocessed so that it is ready for analysis. Specifically, it has been aligned to standard space, making it easy for us to use for masking.

Throughout this course, we are going to be relying on the movie watching (AKA perception) data, rather than the recall data that comes with Sherlock. The data was collected in two segments (AKA runs, since they stopped the scanner at the end of a segment) but they were combined in this dataset. 

**Exercise 2:**<a id="ex2"></a> To make sure you understand the data, we are going to ask you some questions about it. You should either examine the data in the directory or you should read the abstract/methods of the paper to get these answers:

How many participants are there?  
**A:** 

How long was each TR?  
**A:** 

How big was each voxel in millimeters?  
**A:**

How long was the first segment in minutes? (Segment is a specific term from the paper)  
**A:** 

Why is "Let's all go to the lobby" relevant to this dataset?  
**A:**

## 3. The BIDS format <a id="bids"></a>

The [BIDS standard](https://bids.neuroimaging.io) for neuroimaging data is being increasingly used to share data across laboratories. It enables open science practices and reproducibility. Instead of using arbitrary names for files, the BIDS format enforces a file naming structure, making it easy for other people to understand the various files. 

The Sherlock data is stored in a BIDS directory. In the Sherlock directory there are the participant folders, stored by name (e.g., `sub-01`) and in each folder is their data organized by:
>`func`: This folder is where all the functional neuroimaging data (epi) is stored.   
>`anat`: Stores the structural data. 

Keywords that are part of the file name:
>`task`: This label typically describes what the experiment was about.  
>`bold`: This suffix denotes the type of data that is contained in the file. For fmri data it is 'bold'.  

Using the above elements we can construct the file name. Once you have generated the file name, you can use Nilearn's data loading functions to read in the data.


In [ ]:
subj_bids='sub-01' # the subject id of the subject
#Using the above information, we can get the fMRI data file location.
file_loc= os.path.join(sherlock_dir,subj_bids,'func')
print('Directory where files are stored: %s' % file_loc)

#To get a specific file, we will need to add the following.
task_bids='sherlockPart1' # the task name. change this for the specific dataset that you are using.
bids_file_name=  '%s_task-%s' % (subj_bids, task_bids) + '_bold.nii.gz'

print('BIDS file name: %s' % bids_file_name)

bids_full_path_to_file= os.path.join(file_loc, bids_file_name)
print('Full path to file: %s' % bids_full_path_to_file)


**Exercise 3:** <a id="ex3"></a>Explain why a standardized file system is helpful for scientific reproducibility.

**A:**

## 4. Load fMRI data <a id="load_fmri"></a>

We are now going to load in the movie data from a participant for one run. We will then use this data to consider various types of data handling.  As mentioned above, we are not using the raw Sherlock data today, but instead preprocessed data that is stored in the derivatives.

The best way to load and save fMRI data with Python is to use nibabel (aka nib). The first step to load in data is to create an object that contains the header information. We will interrogate that header, and then do the second step: use that object to pull out the data. Below we do both steps:

In [ ]:
# What is the file name? We are getting participant 1
file = sherlock_dir + '/derivatives/movie_files/sub-01.nii.gz'

# Create the nifti object that serves as a header
func_nii = nib.load(file)

# Get the dimensionality of the data
func_dim = func_nii.shape

# Get the functional voxel size
func_voxel = func_nii.header.get_zooms()

# Report some details about the data
print('The dimensionality of the data (x, y, z, TR): ' + str(func_dim))
print('The size of the voxels (x, y, and z are in millimeters, TR is in seconds): ' + str(func_voxel))


<div class="alert alert-block alert-info">
<strong> Recommendation:</strong>  Because Python suppresses output (when there is no error), you may want to include print statements at the end of cells to let you know when a cell has executed all lines of code. This also is a useful technique to debug your programs. 
    
In Jupyter, there is an indicator to show that a cell is running - the asterisk '*' on the left hand side of the cell. Once the cell execution is complete, this changes to a number.
</div>

In [ ]:
# Load the data volume
print('Loading fMRI data, this will take a minute')
func_vol = func_nii.get_fdata()
print('Finished')

<div class="alert alert-block alert-warning">
<strong>Be aware of exceeding your data allocation!</strong> When we started the jupyter notebook you are running, we requested a certain amount of memory resources. This amount is designed to be enough to do various operations, however, you will naturally be prohibited from running data intensive analyses. 
    
fMRI data is high dimensional: there are 150000-300000 voxels per volume, and each voxel is usually stored with single precision 16-bit sign integers. However, numpy works with data in float 64 format, which is even larger. Thus, each voxel 'takes up' 8 bytes of computer memory (you can check this by running `func_vol.itemsize`). 
    
For the data you just loaded, there were 271623 voxels per volume and 1976 TRs, which takes up 4.3 GB when uncompressed. If we loaded several participants and held all the data in memory, we would soon exhaust the data limit. When that happens, the notebook kernel will crash.

Even though this file is 4.3 GB when loaded into memory, the data only takes up 0.9GB when stored in memory. That is because we use the compression codec gzip to compress the file without reducing data quality.    
    
*Fortunately*, you shouldn't lose your progress when you exceed the memory: You can just restart the notebook kernel. Nonetheless be careful loading in lots of fMRI data.  
</div>

**Exercise 4**<a id="ex4"></a>: Visualize an axial slice of the first TR, halfway along the z-axis. If you need help doing this, refer to the code from week 1 where we did something similar

In [ ]:
# Insert code here

The different intensities on this plot reflect the Z-scored intensities for this first volume.

There are lots of great tools for exploring data that aren't just looking at individual slice like you just made. Here is one example tool that uses nilearn, another very useful tool for neuroimaging and Python

In [ ]:
# Get the first TR
print('Loading functional volume to just grab the first TR')
epi_nii = nilearn.image.index_img(file, 0) # the 0 means take just the first TR

# Plot the data as a 3d volume
plotting.plot_stat_map(epi_nii)

That isn't all though! We can make these plots interactive. 

In [ ]:
# Make an interactive plot of your data
plotting.view_img(epi_nii)

In the plot above, you can click on the image and move the crosshairs. In this case, the coordinates at the bottom are the coordinates in standard space, measuring the distance in millimeters from the center of the brain.

We just looked at the data in terms of *space*, so now lets look at the data in *time*. fMRI data is a 4d volume where the 4th dimension is time. This means we can just index a voxel and see the timecourse

**Exercise 5:<a id="ex5"></a>** Plot the timecourse of a voxel. Remember that we loaded the functional data into the 4d matrix `func_vol`. Using that, plot the timecourse of the voxel in the middle of all of the brain (Hint: if the brain were 60 x 80 x 100, plot the timecourse of voxel [30, 40, 50]). Remember to label your axes with meaningful names and the correct units.

In [ ]:
# Insert code here.

## 5. Masking<a id="masking"></a>

One of the key operations we can do on our data is to mask it so that we are only testing the voxels we care about. Functional acquisitions are large volumes that are usually only 30% brain, so it is important we ignore all of the non-brain. Moreover, in our analyses we often care about specific regions of the brain, rather than the brain as a whole.

We are going to examine two ways of masking. The first way is to use numpy tools, treating the data like any array we manipulate with numpy. This is powerful and flexible and good enough for all uses. The second way is more specific to neuroimaging analyses, and comes with a few helpful features that relate to MRI data. 

The masks we are going to use are regions of the visual cortex from the Wang atlas (AKA the Kastner atlas) which comes from [this paper](https://pubmed.ncbi.nlm.nih.gov/25452571/) that is stored in `/farmshare/home/classes/psych/236/atlases/ProbAtlas_v4/`. This atlas divides the visual cortex into about 2 dozen areas (e.g., V1, V2, hMT, IPS1, FEF, etc.). We are working in volume space, so we will use the masks in the `subj_vol_all` folder, but there are also surface masks (surface space is an important format for fMRI data but no one we will cover in this course). For each area there is a volume that is numbered. The legend for the numbers is found in the `ROIfiles_Labeling.txt` file that is in the folder, see below for the output. There is a separate volume for each hemisphere.

In [ ]:
# Print out the legend for the 
!cat $atlas_path/ProbAtlas_v4/ROIfiles_Labeling.txt

<div class="alert alert-block alert-warning">
<strong>Wait a minute!</strong> What just happened in that command above!? It has weird syntax that doesn't look like Python. 
    
Well it wasn't Python, it was shell scripting. Shell script is the language used to interface with linux computers. One of the reasons Jupyter is so cool is that it lets you use shell scripting (or matlab or R) in the same environment as python. Variables even transfer over! 
    
So let us break down the command:<br>
`!`: This tells the notebook that the following command will be in shell script<br>
`cat`: This is a shell script command for printing the content of files.<br>
`$atlas_path`: The `atlas_path` variable is a Python variable (it was made in the `utils` file) and we are able to make the shell script recognize it as a variable by putting the `$` in front. This is really useful!!<br>
`/ProbAtlas_v4/ROIfiles_Labeling.txt`: This is the name of file relative to the path.
</div>



One more detail about masking: the masks from the atlas are in percentiles, meaning that each voxel reports the confidence that the voxel belongs in the mask (read the paper for more details). For masks to be useful, we need them to be binary. In other words, they must denote whether a voxel is in the mask or outside of the mask. Hence we are going to **threshold** the mask and say that any voxels above the threshold are in, any voxels below the threshold are out. For our purpose, our threshold will be 0%, meaning any voxels will be used.

In [ ]:
# Make the mask file
roi_counter = 7 # This is the number that corresponds to the ROI we will use (from the ROIfiles_Labeling file)
mask_threshold = 0 # What is the percentile threshold for the mask that will be applied

# Specify the mask file, this is a left hemisphere
mask_file = '%s/ProbAtlas_v4/subj_vol_all/perc_VTPM_vol_roi%d_lh.nii.gz' % (atlas_path, roi_counter) # This is an important syntax for making strings in python, make sure you understand it

# Load the mask header
mask_nii = nib.load(mask_file)

Visualize the mask using our viewing tools. We are going to set the maximum value to the mask threshold so that the mask is just binary.

In [ ]:
plotting.view_img(mask_nii, 
                  threshold=mask_threshold, 
                  vmax=mask_threshold,
                  colorbar=False,
                 )

### Masking method 1: numpy 
We are now going to apply the mask to our functional data so that we only have voxels from inside the mask. For this method, our mask will be treated as a 3d volume and we will use that to mask a 4d volume. 

In [ ]:
# Get the mask as a volume
mask_vol = mask_nii.get_fdata()

print('Mask dimensionality:', mask_nii.shape)
print('Voxel size:', mask_nii.header.get_zooms())

We just loaded in the mask and looked at the dimensionality. Compare the dimensionality to the dimensionality of the functional data reported [here](#load_fmri).  

Hopefully you will notice that the volume is not the same dimensionality as the functional we loaded before. That means that if we want to make these two arrays interact then we need to change their dimensionality. There are lots of tools for doing this, like FSL, but fortunately nibabel gives us some great tools too. 

nibabel will only work if the data is aligned in the same space but has different voxel sizes. That is is the case for this data (if you want to make sure, divide each dimension by 3 and round a little). You should *NEVER* use these tools when the data is not aligned: the code will *work* but the results will be meaningless.. 

To make nibabel resize the data, we need the dimensionality of the data we want to align the mask to. We also need the voxel size of the data we want to align to. We have already got this information, stored in variables called `func_dim` and `func_voxel`

In [ ]:
# Run the command to align the data. Also see how Python lets you put line breaks within arguments to make it more readible!
mask_aligned_nii = processing.conform(mask_nii, 
                                      out_shape=[func_dim[0], func_dim[1], func_dim[2]], 
                                      voxel_size=(func_voxel[0], func_voxel[0], func_voxel[0]),
                                     )

# Pull the volume data
mask_aligned_vol = mask_aligned_nii.get_fdata()

print('Mask dimensionality:', mask_aligned_nii.shape)

**Self study:** We transformed the mask so that it is the same dimensionality as the functional data, but we could do the opposite: make the functional data the same dimensionality as the mask. Consider how you might do that.

**Exercise 6:<a id="ex6"></a>** Visualize the resized mask using the nilearn viewing tools, in conjunction with the functional data. Specifically, use the EPI as the `bg_img` and the mask as target image. If you aren't sure what `bg_img` is, read the help of the nilearn tools

In [ ]:
# Insert code here

We are going to threshold the mask and report how many voxels are included in the mask

In [ ]:
mask_aligned_vol = mask_aligned_vol > mask_threshold
print('Number of voxels in the mask', mask_aligned_vol.sum())

Now that we have everything, we can mask the functional data. Python makes this very easy, as you will see.

In [ ]:
func_masked = func_vol[mask_aligned_vol == 1] # Take all the voxels equal to one
print('Dimensionality of the data:', func_masked.shape)

You can see that the data is now the number of voxels in the mask by the number of TRs. This is the format we want for masking, and the data is now very usable for other analyses.

To go back to volume space after masking the data, we have to do the same kind of command but in reverse. You might do this when you alter/process the data in some way (e.g., you made a whole brain mask, applied it to the 4d volume to get a voxel by time array, z scored the array and then want to put it back into brain space).

In [ ]:
# This is effectively saying, for all the values in the mask that equal to 1, fill in the corresponding value from the masked data
func_vol[mask_aligned_vol == 1] = func_masked

**Self-study**: What we did in that last step is surprisingly tricky. The mask is 3d but the functional is 4d, yet python, unlike a coding language like Matlab, knew how to interface the two.  This is because of a great feature of numpy, that would be worth the time to explore.

### Masking method 2: nilearn 
This route is a bit different and keeps the tools within nilearn. The first step is to threshold the mask. Then we make the masker object which can be applied to functional data.

In [ ]:
# Load the data via nliearn (which is different but related to nibabel)
mask_img = nilearn.image.load_img(mask_aligned_nii)

# Threshold the data using nibabel math (which is a bit clunky)
mask_img = nilearn.image.math_img('img > %s' % mask_threshold, img=mask_img)

In [ ]:
# Finalize the masker object
nifti_masker = NiftiMasker(mask_img=mask_img)

In [ ]:
# Apply the masker object to the nifti data, this will take some time. 
print('Masking nifti, this will take a minute')
func_masked = nifti_masker.fit_transform(func_nii)

print('Dimensionality of the data:', func_masked.shape)

# You can even do this all in one step, like the code below:
#func_masked = nilearn.masking.apply_mask(func_nii, mask_img)

You can see the data is the same dimensionality but transposed, a quirk of this method to be wary of!

To go back into volume space, you can use the command below. 

In [ ]:
#Unmask the data
unmasked_func = nilearn.masking.unmask(func_masked, mask_img)

print('Dimensionality of the data:', func_masked.shape)
print('Number of non-zero voxels per volume: %d' % np.sum(unmasked_func.get_fdata()[:, :, :, 0] != 0))

**Note:** due to the way nilearn works, this only puts the functional data into volume space, it doesn't put it back into the volume it came from. In other words, the only voxels in the volume are those from the mask. This is why there are only 548 non-zero voxels in this new mask

**Exercise 7:<a id="ex7"></a>** Describe one reason or scenario you think the first masking method (numpy) is better, and another scenario where you think the second (nilearn) is better. There are lots of answers here, but you will need to think carefully!

*Hint: for one of the reasons, consider memory demands.*

**A:**

Masking is a surprisingly important skill to have, so you will see that we will return to it a few times throughout the course. Make sure you understand it well now.

## 6. Homotopic analyses<a id="homotopy"></a>

So we have done all these steps for data handling, lets now do something cool with it! Specifically, we are going to test the degree to which the functional activity is similar in the same visual region across areas. This homotopic analysis was originally introduced by [Biswal and colleagues (1995)](https://onlinelibrary.wiley.com/doi/10.1002/mrm.1910340409) and serves as a good test of whether functional activity is due to noise or is driven by signal. This is because the regions are separated by anatomical distance, so any similarity is likely due to shared driving activity.

To do this we are going to mask the left and right side of a visual region (in our case, hV4) and then apply that mask to our functional data. This gives us two arrays of voxel by time. We will then average across voxels to get a timecourse of activity. Finally, we will correlate the two timecourses to see how related they are.

In [ ]:
# Specify the ROI you are using
roi_counter = 7

hemi_timecourses = {} # Preset array to store the data
plt.figure() #Set up the figure
for hemi in ['lh', 'rh']:
    
    # Specify the ROI name you will use to mask
    roi_name = 'roi%d_%s' % (roi_counter, hemi)
    
    # Specify the mask file, this is a left hemisphere
    mask_file = '%s/ProbAtlas_v4/subj_vol_all/perc_VTPM_vol_%s.nii.gz' % (atlas_path, roi_name) # This is an important syntax for making strings in python, make sure you understand it

    # Load the mask header
    mask_nii = nib.load(mask_file)
    
    # Run the command to align the data to our functional
    mask_aligned_nii = processing.conform(mask_nii, 
                                          out_shape=[func_dim[0], func_dim[1], func_dim[2]], 
                                          voxel_size=(func_voxel[0], func_voxel[0], func_voxel[0]),
                                         )

    # Pull the mask data and threshold it
    mask_aligned_vol = mask_aligned_nii.get_fdata() > mask_threshold
    
    # Mask the functional data
    func_masked = func_vol[mask_aligned_vol == 1]
    
    # Average the data along the voxel axis
    hemi_timecourse = np.mean(func_masked, axis=0)
    
    # Plot the first 100 TRs of the timecourse
    plt.plot(hemi_timecourse[:100])
    
    # Store the timecourse
    hemi_timecourses[roi_name] = hemi_timecourse
    
# Specify the figure parameters    
plt.xlabel('TR')
plt.ylabel('Intensity (Z)')
plt.title('Similarity of timecourse between hemispheres for the first 100 TRs')
plt.legend(['left hemi', 'right hemi']);

In [ ]:
# Report the similarity of the timecourses
lh_timecourse = hemi_timecourses['roi%d_lh' % roi_counter]
rh_timecourse = hemi_timecourses['roi%d_rh' % roi_counter]

r_val = np.corrcoef(lh_timecourse, rh_timecourse)[0, 1]

print('Correlation between left and right hemisphere for ROI %d: %0.2f' % (roi_counter, r_val))


Woohoo! That is a pretty high correlation! Even though the left and right hemispheres are far apart, the fluctuations in activity between the left and right hV4 are almost identical! This is a good sign that these regions are responding strongly to the movie. So even though those fluctuations look kind of noisy, they seem to contain real signal related to the participant watching the movie. In a way, this is the very essence of the course and shows the potential that movies have for fMRI analysis.

**Exercise 8:<a id="ex8"></a>** Hold on a minute! This analysis is misleading because it suggests that the two regions are highly correlated *because* of the movie, but they could be correlated for other reasons. For instance, these regions might have a high correlation at rest when there is no movie, suggesting that the movie isn't evoking activity. Another reason this correlation could be misleading is that motion could be a confound. Speculate on a way that you think motion could cause this correlation. 

**A:**

**Exercise 9<a id="ex9"></a>:** One way to test whether this correlation is capturing signal is to compare this correlation we just got with a non-homotopic correlation. For instance, you could correlate left LO1 with right hV4 to test if that non-homotopic correlation is lower. 

In the code below, report the correlation between right hV4 and left LO1. Then in the section after it, interpret that correlation (i.e., why this is informative?)

In [ ]:
# Insert code here

**Interpret results here:**

## Novel contribution

You have now finished the main part of the problem set, but now it is time for extra credit. Remember that you can get up to 10% more on your problem set grade by completing an **optional** novel contribution. To give you an idea of what you could do, focus on *part a* of the figure below. This is from [Arcaro and Livingstone (2017)](https://elifesciences.org/articles/26196) and reports a homotopy analysis in which several regions from the left hemisphere are compared to each other, forming a matrix (shown on the left). Although it seems complicated, the code above only needs to be edited a little to make this kind of plot.

If you make a novel contribution that involves this level of insight or creativity then you will be rewarded with extra credit. However, you should know that fewer than half of the class will get extra credit.

<img src=https://iiif.elifesciences.org/lax/26196%2Felife-26196-fig2-v1.tif/full/1500,/0/default.jpg >

In [ ]:
# Put novel contribution here

## Contributions <a id="contributions"></a>

M. Kumar, C. Ellis and N. Turk-Browne produced the initial notebook 01/2018  
T. Meissner minor edits  
Q. Lu switch to matplotlib, color blind friendly colors, encapsulate helper functions, del ex.3 (loop)  
M. Kumar: Added Exercise 10, deleted masking exercise.  
K.A. Norman provided suggestions on the overall content and made edits to this notebook.  
C. Ellis: Incorporated comments from cmhn-s19.   
A.K. Sahoo made minor edits to the notebook.  
Q. Lu consistent plotting format; slightly modified the exs; adding solutions.  
T. Yates edits for cmhn-s21  
C. Ellis edits for mmn23